In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
import geopandas as gpd
from collections import defaultdict
from joblib import Parallel, delayed
from datetime import datetime, timedelta
import time

path = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/'
path_b2 = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/vrt/256_20_chips.vrt'

In [2]:
# load the polygons made from prediction
path_fields = f'{path}Fields_polygons.shp'

# Convert polygons to lines
gdf = gpd.read_file(path_fields)

# Ensure the geometries are polygons
if not all(gdf.geometry.type.isin(["Polygon", "MultiPolygon"])):
    raise ValueError("The file must contain Polygon or MultiPolygon geometries.")


gdf['geometry'] = gdf.geometry.boundary
#export
gdf.to_file(f'{path}median_scores/Field_lines.gpkg', driver='GPKG') 

In [3]:
##### open field vector file
field = ogr.Open(f'{path}median_scores/Field_lines.gpkg')
field_lyr = field.GetLayer(0)

output = f'{path}median_scores/FieldBorders.tif'

if not os.path.exists(output):
    ds = gdal.Open(path_b2)
    target_ds = gdal.GetDriverByName('GTiff').Create(output, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_UInt32)
    target_ds.SetGeoTransform(ds.GetGeoTransform())
    target_ds.SetProjection(ds.GetProjection())

    gdal.RasterizeLayer(target_ds, [1], field_lyr, options = ["ALL_TOUCHED=True", "ATTRIBUTE=FieldID"])
    target_ds = None

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
# extract b2 at rasterized borders
ds_b2 = gdal.Open(path_b2)
arr_b2 = ds_b2.GetRasterBand(2).ReadAsArray()

ds_border = gdal.Open(output)
arr_border = ds_border.GetRasterBand(1).ReadAsArray()

mask = (arr_border != 0) & (~np.isnan(arr_border)) & (~np.isnan(arr_b2))
flat_field = arr_border[mask].astype(int)
flat_b2 = arr_b2[mask]

In [5]:
# Create DataFrame
df = pd.DataFrame({'BorderID': flat_field, 'B2': flat_b2})

# Compute median per field
median_df = df.groupby('BorderID', as_index=False)['B2'].median()
median_df.rename(columns={'B2': 'B2_median'}, inplace=True)
median_df.to_csv('mean_b2_score_per_field.csv', index=False)

In [7]:
# Create a lookup map
median_map = dict(zip(median_df['BorderID'], median_df['B2_median']))

# Fill raster
arr_border_fill = np.vectorize(median_map.get)(arr_border)

In [ ]:
npTOdisk(arr_border_fill, output,
         '/data/Aldhani/eoagriwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/median_scores/Borders_MEDIAN_B2_SCORE.tif',
         noData=0, d_type=gdal.GDT_Float32)